# Clonal network representation for tracer output

## Import the needed packages

In [ ]:
import pandas as pd
from pyvis.network import Network
import matplotlib.pyplot as plt
import numpy as np


## Get the color dictionary

In [ ]:
def color_dict():
    tissues = ['Bladder','Brain_Myeloid','Brain_Non-Myeloid','Fat','Heart_and_Aorta','Kidney','Large_Intestine','Limb_Muscle','Liver','Lung','Mammary_Gland','Marrow','Pancreas','Skin','Spleen','Thymus','Tongue','Trachea','BAT', 'GAT', 'MAT', 'SCAT']
    
    import matplotlib.colors as pltcolors
    cmap = plt.cm.get_cmap("tab20b")

    color_dict = {}
    i=0
    for t in tissues:
        color_dict[t] = pltcolors.to_hex(cmap(i))
        i+=1/len(tissues)
    return color_dict

maca_color_dict = color_dict()

## Parse the tracer output with metadata

In [ ]:
tracer = pd.read_csv('../../0_data_ingest/T-cell-clonality/Supplementary_Table_6_T_cells.csv')
tracer['validated_clone'] = 999
tracer['validated_clone_length'] = 1.0

clonegroup = list(set(tracer.clonal_group[~tracer['clonal_group'].isnull()]))
for c in clonegroup:
    cg = tracer.loc[tracer['clonal_group']==c]
    mice = set(cg['mouse.id'])
    if len(mice) == 1:
        for i in cg.index:
            tracer.loc[i,'validated_clone'] = 1
            tracer.loc[i,'validated_clone_length'] = len(cg)
    else:
        miceidx = pd.DataFrame(cg['mouse.id'].value_counts())
        im = 0
        for idxmice in miceidx.index:
            im = im+1
            for i in cg.index:
                if cg.loc[i,'mouse.id']==idxmice:
                    tracer.loc[i,'clonal_group'] = str(c)+'_'+str(im)
                    tracer.loc[i,'validated_clone'] = 2
            
validatedclones2 = pd.DataFrame(tracer.loc[tracer['validated_clone']==2].groupby(['clonal_group']).size())
for vc2 in validatedclones2.index:
    if validatedclones2.loc[vc2][0]>1:
        cg = tracer.loc[tracer['clonal_group']==vc2]
        for i in cg.index:
            tracer.loc[i,'validated_clone'] = 1
            tracer.loc[i,'validated_clone_length'] = validatedclones2.loc[vc2][0]
            tracer.loc[i,'clonal_group'] = float(tracer.loc[i,'clonal_group'].split('_')[0])
    else:
        cg = tracer.loc[tracer['clonal_group']==vc2]
        for i in cg.index:
            tracer.loc[i,'validated_clone'] = 999
            tracer.loc[i,'validated_clone_length'] = 1.0

In [ ]:
tracer_validated = tracer[tracer['validated_clone']==1]
tracer_validated = tracer_validated.replace('Heart','Heart_and_Aorta')

## Generate the network for the clonal families

In [ ]:
G = Network(height=2000,width=1000,notebook=True)

edgelist = []
vertice = []
for i in tracer_validated.index:
        
    edge = (str(tracer_validated.loc[i, 'age']), tracer_validated.loc[i, 'mouse.id'])
    if edge not in edgelist:
        vert1 = str(tracer_validated.loc[i, 'age'])
        if vert1 not in vertice:
            G.add_nodes([vert1],label = [tracer_validated.loc[i, 'age']], color = ['black'],#color = ["#dd4b39"],
                       value = [10])
            vertice.append(str(vert1))
        vert2 = tracer_validated.loc[i, 'mouse.id']
        if vert2 not in vertice:
            G.add_nodes([vert2], color = ["#555555"],label = [tracer_validated.loc[i, 'mouse.id']])
            vertice.append(vert2)
        G.add_edge(edge[0],to=edge[1])
        edgelist.append(edge)
        
    edge = (tracer_validated.loc[i, 'mouse.id'], tracer_validated.loc[i, 'clonal_group'])
    if edge not in edgelist:
        vert1 = tracer_validated.loc[i, 'mouse.id']
        if vert1 not in vertice:
            G.add_nodes([vert1],label = [tracer_validated.loc[i, 'mouse.id']], color = ["#555555"])#
            vertice.append(str(vert1))
        vert2 = tracer_validated.loc[i, 'clonal_group']
        if vert2 not in vertice:
            G.add_nodes([vert2], color = ["#737373"],label = [' '],
                       value = [tracer_validated.loc[i, 'validated_clone_length']])
            vertice.append(vert2)
        G.add_edge(edge[0],to=edge[1])
        edgelist.append(edge)
        
    edge = (tracer_validated.loc[i, 'cell'], tracer_validated.loc[i, 'clonal_group'])
    if edge not in edgelist:
        vert1 = tracer_validated.loc[i, 'cell']
        if vert1 not in vertice:
            G.add_nodes([vert1],label = [' '], color = [maca_color_dict[tracer_validated.loc[i, 'tissue']]])
            vertice.append(str(vert1))
        vert2 = tracer_validated.loc[i, 'clonal_group']
        if vert2 not in vertice:
            G.add_nodes([vert2], color = ["#737373"],label = [tracer_validated.loc[i, 'clonal_group']],
                       value = [tracer_validated.loc[i, 'validated_clone_length']])
            vertice.append(vert2)
        G.add_edge(edge[0],to=edge[1])
        edgelist.append(edge)
        
G.show('tracer_validated.html')



## Generate the network for the whole output

These might take a really long time...

In [ ]:
G = Network(height=2000,width=1000,notebook=True)

edgelist = []
vertice = []
vidx = int(np.nanmax(tracer['clonal_group'].values.astype(float)))+1.
for i in tracer.index:
        
    edge = (str(tracer.loc[i, 'age']), tracer.loc[i, 'mouse.id'])
    if edge not in edgelist:
        vert1 = str(tracer.loc[i, 'age'])
        if vert1 not in vertice:
            G.add_nodes([vert1],label = [tracer.loc[i, 'age']], color = ["#dd4b39"],
                       value = [10])
            vertice.append(str(vert1))
        vert2 = tracer.loc[i, 'mouse.id']
        if vert2 not in vertice:
            G.add_nodes([vert2], color = ["#00ff1e"],label = [tracer.loc[i, 'mouse.id']])
            vertice.append(vert2)
        G.add_edge(edge[0],to=edge[1])
        edgelist.append(edge)
        
    edge = (tracer.loc[i, 'mouse.id'], tracer.loc[i, 'clonal_group'])
    if edge not in edgelist:
        vert1 = tracer.loc[i, 'mouse.id']
        if vert1 not in vertice:
            G.add_nodes([vert1],label = [tracer.loc[i, 'mouse.id']], color = ["#00ff1e"])
            vertice.append(str(vert1))
        vert2 = tracer.loc[i, 'clonal_group']
        if vert2 not in vertice:
            if math.isnan(float(vert2)):
                vert2 = vidx
                val = 1.0
                vidx = vidx+1
            else:
                val = tracer.loc[i, 'validated_clone_length']
            G.add_nodes([vert2], color = ["#FFD700"],label = [' '],
                       value = [val])
            vertice.append(vert2)
        G.add_edge(edge[0],to=vert2)
        edgelist.append(edge)
        
    edge = (tracer.loc[i, 'cell'], tracer.loc[i, 'clonal_group'])
    if edge not in edgelist:
        vert1 = tracer.loc[i, 'cell']
        if vert1 not in vertice:
            G.add_nodes([vert1],label = [' '], color = ["#162347"])
            vertice.append(str(vert1))
        vert2 = tracer.loc[i, 'clonal_group']
        if vert2 not in vertice:
            if math.isnan(float(vert2)):
                vert2 = vidx
                val = 1.0
                vidx = vidx+1
            else:
                val = tracer.loc[i, 'validated_clone_length']
            G.add_nodes([vert2], color = ["#FFD700"],label = [tracer.loc[i, 'clonal_group']],
                       value = [val])
            vertice.append(vert2)
        G.add_edge(edge[0],to=vert2)
        edgelist.append(edge)
        
G.save_graph('tracer.html')

